# XX_threshold_50_data.ipynb

## 閾値除去（閾値50μV）の実施

## .datファイルの作成、個人別の波形出力

### 使用脳波データは、XX_data内のdatファイル

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
files = ["filename1", "filename2", "filename3",]

### 1. datファイルからのデータロード、閾値除去の実施

### data_EEG = 
### image_stimulus(HLでソート)
### × channel(AF3, AFz, AF4, F3, Fz, F4) 
### × frame(500fpsなので、550frames)

### label =  
### image_stimulus(HLでソート)
### × HL

In [ ]:
file = 0
with open(f'../dataset/XX_data/{files[file]}_20_epochs_pruned_with_ICA_baseline.dat', mode='rb') as f:
        data_EEG = pickle.load(f)
        label = pickle.load(f)

# 閾値除去データのデータフレーム用意
columns = ['image_stimulus', 'HL']
threshold_50_data = pd.DataFrame(data=label, columns=columns)
threshold_50_data

In [ ]:
ch_name = ['AF3', 'AFz', 'AF4', 'F3', 'Fz', 'F4']
color = ["red", "green", "blue", "purple", "orange", "pink"]


for file in range(len(files)):
    
    # 脳波データ(data_EEG), ラベル（label)のロード
    
    with open(f'../dataset/XX_data/{files[file]}_20_epochs_pruned_with_ICA_baseline.dat', mode='rb') as f:
        data_EEG = pickle.load(f)
        label = pickle.load(f)
     
    # 全チャネルで、脳波の絶対値が50μVを超えるframeがあるエポックの、配列の値を全てnanにする
    
    for image in range(len(label)):
        for frame in range(550):
            if np.all(np.abs(data_EEG[image,:,frame])>50):
                data_EEG[image,:,:] = np.nan
            
            
    # 除去された（nanにされた）データの一覧作成
    
    threshold_50_data[f'{files[file]}_50'] = np.all(np.all(np.isnan(data_EEG), axis = 1), axis=1)
    
    
    # グラフ化（nanにしたデータがグラフ化されない）  、グラフの保存     
    
    fig = plt.figure(figsize = (300,200))
    plt.suptitle(f'{files[file]}(threshold_50)', fontsize=60)
    
    for image in range(len(label)):
        plt.subplot(16, 17 , image+1)
        plt.title(f'{label[image,0]}_{label[image,1]}', fontsize=50)

        x = np.linspace(-100, 1000, 550)
        for c in range(6):
            y = data_EEG[image, c,:]
            plt.plot(x, y, lw=3, color = color[c], label=ch_name[c])

        plt.ylim(-50, 50)
        plt.xlim(-100, 1000)
        plt.xlabel('Time (ms)',fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel('Potential (μV)',fontsize=20)
        plt.legend(fontsize=20, loc='upper right')
        plt.yticks(fontsize=20)

        plt.hlines(y=0, xmin=-100, xmax=1000, color = 'k')
        plt.vlines(x=0, ymin=-100, ymax=1000, color = 'k')
        plt.grid()

    plt.show()
    fig.savefig(f'../dataset/XX_threshold_50_data/{files[file]}_threshold_50_data.png')
    
    
    # 閾値除去後の脳波データ(data_EEG), ラベル（label)の保存

    with open(f'../dataset/XX_threshold_50_data/{files[file]}_threshold_50.dat', mode='wb') as f:
        pickle.dump(data_EEG, f)
        pickle.dump(label, f)

    print(f"{files[file]}_threshold_50.dat", "complete")
    print("data_EEG: ", data_EEG.shape)
    print("label: ", label.shape)
    
    print(f"{files[file]}_threshold_50", "complete")
    
# 閾値除去データに、除去数のトータルを入力、データフレームの入力

with open(f'../dataset/XX_threshold_50_data/threshold_50.dat', mode='wb') as f:
        pickle.dump(threshold_50_data, f)


In [ ]:
threshold_50_data.sum()